In [7]:
import folium
import openrouteservice as ors
import math
from geopy.distance import geodesic
import pandas as pd
import time

#Enter your ORS API key to run this model
client = ors.Client(key='...')

In [8]:
# Functions
def swap_lat_lon(coords):
    """
    Swap latitude and longitude in a list of coordinates.

    Parameters:
    coords (list): A list of coordinates where each coordinate is a list [lat, lon].

    Returns:
    list: A list of coordinates with swapped latitude and longitude.
    """
    return [[lon, lat] for lat, lon in coords]

# Original list of coordinates
#coords = [[10.78017232473696, 106.63830134919867], [10.766216, 106.640663]]
#
## Swapping latitude and longitude using the function
#swapped_coords = swap_lat_lon(coords)
#
#print(swapped_coords)

def visualize_coordinates_on_map(coords_list):
    """
    Visualize a list of coordinates on a map using Folium.

    Parameters:
    coords_list (list): A list of coordinates where each coordinate is a list [lat, lon].

    Returns:
    folium.Map: A Folium map with the coordinates plotted.
    """
    # Create a map centered at the first coordinate
    m = folium.Map(location=coords_list[0], tiles="cartodbpositron", zoom_start=14)
    
    # Add markers for each coordinate
    for coord in coords_list:
        folium.Marker(location=coord, popup=f"Coordinate: {coord}").add_to(m)
    
    return m

# Example usage
#coords_list = [[10.78017232473696, 106.63830134919867], [10.766216, 106.640663]]
#map_visualization = visualize_coordinates_on_map(coords_list)
#map_visualization



# Function to apply a small offset to coordinates
def apply_offset(coords, offset):
    return [[coord[0] + offset, coord[1] + offset] for coord in coords]

In [9]:
# Import data into Dataframe
df = pd.read_excel(r'C:\Users\anhpd\OneDrive\Desktop\MY House Data\Coordinates.xlsx')
df

,location1,Coordinate,AVERAGE của price,AVERAGE của m2,AVERAGE của rm_per_space
0,"1120 Park Avenue, Petaling Jaya, Malaysia","3.0776657641473615, 101.64785284054406",1700.000000,870.000000,1.950000
1,"216 Residences, Kuchai Lama, Malaysia","3.0911898820468235, 101.68255818287214",2800.000000,1023.000000,2.740000
2,"288 Residences, Jalan Klang Lama (Old Klang Ro...","3.0809516435129587, 101.68803758102025",2500.000000,1215.000000,2.060000
3,"51 Boulevard, Petaling Jaya, Malaysia","3.087703229331669, 101.62337092960392",2250.000000,1000.000000,2.250000
4,"Acappella Residences, Shah Alam, Malaysia","3.0889916871113, 101.54783642705208",2366.666667,884.666667,2.706667
...,...,...,...,...,...
272,"Vistaria, Puchong, Malaysia","3.001828993400486, 101.60052189821587",1166.666667,826.666667,1.416667
273,"Vivo Residential Suites, Jalan Klang Lama, Jal...","3.105807610753234, 101.67726048102034",2635.035088,1053.157895,2.561579
274,"Vogue Suites One, KL Eco City, Malaysia","3.1170854561311936, 101.67415918472427",3000.000000,784.666667,3.823333
275,"Waltz Residences, Taman Overseas Union, Jalan ...","3.0688337928341083, 101.67235911170812",2550.000000,1021.000000,2.507500


In [31]:
# Split 'Coordinate' column into 'lat' and 'lon', ensuring valid float conversion
df[['lat', 'lon']] = df['Coordinate'].str.split(',', expand=True).astype(float)

# Define the reference point (latitude, longitude)
reference_point = (3.0672856209707606, 101.60385172520009)

# Calculate distances from reference point, rounded to two decimal places
df['distance_to_reference (m)'] = df.apply(
    lambda row: round(geodesic(reference_point, (row['lat'], row['lon'])).meters, 2), axis=1
)
df

,location1,Coordinate,AVERAGE của price,AVERAGE của m2,AVERAGE của rm_per_space,lat,lon,distance_to_reference (m)
0,"1120 Park Avenue, Petaling Jaya, Malaysia","3.0776657641473615, 101.64785284054406",1700.000000,870.000000,1.950000,3.077666,101.647853,5024.06
1,"216 Residences, Kuchai Lama, Malaysia","3.0911898820468235, 101.68255818287214",2800.000000,1023.000000,2.740000,3.091190,101.682558,9139.58
2,"288 Residences, Jalan Klang Lama (Old Klang Ro...","3.0809516435129587, 101.68803758102025",2500.000000,1215.000000,2.060000,3.080952,101.688038,9479.36
3,"51 Boulevard, Petaling Jaya, Malaysia","3.087703229331669, 101.62337092960392",2250.000000,1000.000000,2.250000,3.087703,101.623371,3131.32
4,"Acappella Residences, Shah Alam, Malaysia","3.0889916871113, 101.54783642705208",2366.666667,884.666667,2.706667,3.088992,101.547836,6673.25
...,...,...,...,...,...,...,...,...
272,"Vistaria, Puchong, Malaysia","3.001828993400486, 101.60052189821587",1166.666667,826.666667,1.416667,3.001829,101.600522,7247.48
273,"Vivo Residential Suites, Jalan Klang Lama, Jal...","3.105807610753234, 101.67726048102034",2635.035088,1053.157895,2.561579,3.105808,101.677260,9204.95
274,"Vogue Suites One, KL Eco City, Malaysia","3.1170854561311936, 101.67415918472427",3000.000000,784.666667,3.823333,3.117085,101.674159,9560.47
275,"Waltz Residences, Taman Overseas Union, Jalan ...","3.0688337928341083, 101.67235911170812",2550.000000,1021.000000,2.507500,3.068834,101.672359,7617.27


In [32]:
# Filter out top 70 closest points by distance
dff = df.sort_values('distance_to_reference (m)', ascending=True).reset_index().head(70)

# Input School Coordinates
school_coordinate = [3.0672856209707606, 101.60385172520009]

# Clean data to input into data model
point_A_coord = dff[['lon', 'lat']].values.tolist()
point_B_coord = swap_lat_lon([school_coordinate])[0]
name_list = dff['location1'].values.tolist()

# Initialize lists for optimization results
duration1way = []
distance2way = []
geometry__ = []
names = []

# Create vehicles and jobs for optimization
for index, point in enumerate(point_A_coord):
    # Define a vehicle and a job for the optimization model
    vehicle = ors.optimization.Vehicle(id=index, profile='driving-car', start=point, end=point, capacity=[1])
    job = ors.optimization.Job(id=index, location=point_B_coord, amount=[1])

    # Run the optimization
    optimized = client.optimization(jobs=[job], vehicles=[vehicle], geometry=True)

    # Extract and append optimization results
    route = optimized['routes'][0]
    duration1way.append(round(route['duration'] / 60, 2))  # Duration (minutes)
    distance2way.append(route['distance'])                 # Distance (meters)
    geometry__.append(route['geometry'])                   # Route geometry codes
    names.append(name_list[index])                         # Store the corresponding location name

    # Log completion of each route calculation
    print(f"Completed route {len(duration1way)}")

Completed route 1
Completed route 2
Completed route 3
Completed route 4
Completed route 5
Completed route 6
Completed route 7
Completed route 8
Completed route 9
Completed route 10
Completed route 11
Completed route 12
Completed route 13
Completed route 14
Completed route 15
Completed route 16
Completed route 17
Completed route 18
Completed route 19
Completed route 20
Completed route 21
Completed route 22
Completed route 23
Completed route 24
Completed route 25
Completed route 26
Completed route 27
Completed route 28
Completed route 29
Completed route 30
Completed route 31
Completed route 32
Completed route 33
Completed route 34
Completed route 35
Completed route 36
Completed route 37
Completed route 38
Completed route 39
Completed route 40


C:\Users\anhpd\AppData\Roaming\Python\Python310\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\anhpd\AppData\Roaming\Python\Python310\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\anhpd\AppData\Roaming\Python\Python310\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
C:\Users\anhpd\AppData\Roaming\Python\Python310\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Completed route 41
Completed route 42
Completed route 43
Completed route 44
Completed route 45
Completed route 46
Completed route 47
Completed route 48
Completed route 49
Completed route 50
Completed route 51
Completed route 52
Completed route 53
Completed route 54
Completed route 55
Completed route 56
Completed route 57
Completed route 58
Completed route 59
Completed route 60
Completed route 61
Completed route 62
Completed route 63
Completed route 64
Completed route 65
Completed route 66
Completed route 67
Completed route 68
Completed route 69
Completed route 70


In [39]:
#Create road_df dataframe contain created route data
road_df = pd.DataFrame({
    'duration1way': duration1way,
    'distance2way': distance2way,
    'geometry': geometry__, 
    'name': names
})

#Check road_df
road_df

,duration1way,distance2way,geometry,name
0,3.67,1460,exvQsrckRAd@At@?JCbCAnB?F?TdEB^?nB@`@?FGHQ?o@@...,"Indah Villa Condominium, Bandar Sunway, Malaysia"
1,7.33,2776,cuvQkdbkR_@AsB@?pBbF?\??O?QAiBn@?dC@lC?xB?X?`B...,"The Grand Subang Jaya SS13, Subang Jaya, Malaysia"
2,17.28,9236,yltQcsckRxBLB@BjG^E?mIR??~@?fL?zCBvCBbJZ`@KJOT...,"The New Duo @ Edumetro, USJ 1, Subang Jaya, Ma..."
3,15.85,11206,kmtQauckRVBzC@R??~@?fL?zCBvCBbJZ`@KJOT?|A?z@?h...,The Duo@Subang Jaya USJ1 SEGI COLLEGE SUBANG J...
4,17.27,9236,smtQeockRX}BxBLB@BjG^E?mIR??~@?fL?zCBvCBbJZ`@K...,"Lakeview Suites @ Edumetro, USJ 1, Subang Jaya..."
...,...,...,...,...
65,18.83,13315,ukrQso}jR?MAcBe@??[iA?oQ?BuM?k@cNB{AI_AQqBu@w@...,"USJ 3, USJ, Malaysia"
66,28.28,18334,ubwQej|jRbAb@PF`Bv@RFPHdA^zAZf@L^Lf@JXJdB|@jCb...,"Paisley @ Tropicana Metropark, Tropicana Metro..."
67,16.02,13577,u_}QwkakR}BmADM|C|A|@b@nHfEpB`Ar@^bIdEpAr@ZLRB...,"Kelana Mahkota, Kelana Jaya, Malaysia"
68,20.52,12730,yjoQqo`kREVf@FzB`@nA^~@|@b@f@XXUf@IXANXt@Nn@^v...,"The 19 Usj City Mall, Usj, USJ, Malaysia"


In [43]:
# Merge road_df with dff on the 'name' and 'location1' columns
merged_df = road_df.merge(dff, left_on='name', right_on='location1')

# Round specified columns to 2 decimal places
merged_df[['AVERAGE của price', 'AVERAGE của m2', 'AVERAGE của rm_per_space']] = merged_df[['AVERAGE của price', 'AVERAGE của m2', 'AVERAGE của rm_per_space']].round(2)

# Check if the length of merged_df matches the lengths of the two dataframes
len(merged_df)

70

In [45]:
# Cell to compare up to 4 routes; add more colors in line_colors if comparing more routes
# Sort merged_df by 'distance2way' in ascending order, and select the first 3 rows
df2 = merged_df.sort_values('distance2way', ascending=True).head(3)

# Create a map centered at the school coordinates (point B)
map_routes = folium.Map(location=list(reversed(point_B_coord)), tiles="cartodbpositron", zoom_start=14)

# Add marker for the school location (point B)
folium.Marker(
    location=list(reversed(point_B_coord)),
    popup="School Location",
    icon=folium.Icon(color="red")
).add_to(map_routes)

# Add markers for points A (potential house locations)
for lat, lon, name in df2[['lat', 'lon', 'location1']].values:
    folium.Marker(
        location=[lat, lon],
        popup=f"Location: {name}",
        icon=folium.Icon(color="blue")
    ).add_to(map_routes)

# Colors to visualize individual routes; ensure enough colors for the number of routes
line_colors = ['green', 'orange', 'blue', 'yellow']

# Offset value to separate overlapping routes visually on the map
offset_value = 0.00001

# Add the routes to the map with different colors
for i, route in enumerate(df2['geometry']):
    print(f"Processing route {i+1}/{len(df2['geometry'])}")
    
    # Decode the polyline geometry into coordinates
    decoded_route = ors.convert.decode_polyline(route)['coordinates']
    
    # Apply an offset to the route coordinates to avoid overlapping routes
    offset_route = apply_offset(decoded_route, i * offset_value)
    
    # Add the polyline to the map with color corresponding to the route index
    folium.PolyLine(
        locations=[list(reversed(coord)) for coord in offset_route], 
        color=line_colors[i]
    ).add_to(map_routes)
    
    print(f'Route {i+1} added successfully')

# Display the map
map_routes


Processing route 1/3
Route 1 added successfully
Processing route 2/3
Route 2 added successfully
Processing route 3/3
Route 3 added successfully


In [30]:
# Cell to visualize routes based on geometry code (unlimited number of routes)
# Filter and sort merged_df by rows where 'duration1way' is greater than 15 minutes
df3 = merged_df[merged_df['duration1way'] > 15].sort_values('distance2way', ascending=True).reset_index()

# Create a map centered at the school coordinate (point B)
map_routes = folium.Map(location=list(reversed(point_B_coord)), tiles="cartodbpositron", zoom_start=14)

# Add marker for the school location (point B)
folium.Marker(
    location=list(reversed(point_B_coord)),
    popup="School Location",
    icon=folium.Icon(color="red")
).add_to(map_routes)

# Add markers for points A (potential house locations)
for lat, lon, name in df3[['lat', 'lon', 'location1']].values:
    folium.Marker(
        location=[lat, lon],
        popup=f"Location: {name}",
        icon=folium.Icon(color="blue")
    ).add_to(map_routes)

# Add and visualize each route with a default color
for i, route in enumerate(df3['geometry']):
    print(f"Processing route {i+1}/{len(df3['geometry'])}")
    
    # Decode the polyline geometry into coordinates
    decoded_route = ors.convert.decode_polyline(route)['coordinates']
    
    # Add the polyline to the map (color can be adjusted if necessary)
    folium.PolyLine(
        locations=[list(reversed(coord)) for coord in decoded_route],
        color='green'
    ).add_to(map_routes)
    
    print(f'Route {i+1} added successfully')

# Display the map
map_routes


Processing route 1/45
Route 1 added successfully
Processing route 2/45
Route 2 added successfully
Processing route 3/45
Route 3 added successfully
Processing route 4/45
Route 4 added successfully
Processing route 5/45
Route 5 added successfully
Processing route 6/45
Route 6 added successfully
Processing route 7/45
Route 7 added successfully
Processing route 8/45
Route 8 added successfully
Processing route 9/45
Route 9 added successfully
Processing route 10/45
Route 10 added successfully
Processing route 11/45
Route 11 added successfully
Processing route 12/45
Route 12 added successfully
Processing route 13/45
Route 13 added successfully
Processing route 14/45
Route 14 added successfully
Processing route 15/45
Route 15 added successfully
Processing route 16/45
Route 16 added successfully
Processing route 17/45
Route 17 added successfully
Processing route 18/45
Route 18 added successfully
Processing route 19/45
Route 19 added successfully
Processing route 20/45
Route 20 added successfull

In [17]:
## Save the merged DataFrame to an Excel file
#merged_df.to_excel("sunway_house_coordinates_merged.xlsx", index=False)